In [46]:
import spatialite
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [47]:
xml_path = '/home/aubrey/Desktop/20201001/cvat_annotation_20201001_092426_skip_5_numframes_30315.xml'
gps_path = '/home/aubrey/Desktop/20201001/20201001_092426_gps.csv'
db_path = '/home/aubrey/Documents/roadside/spatialite/Guam01.db'
video_path = '/home/aubrey/Desktop/20201001/20201001_092426.mp4'

In [48]:
# Get frame_numbers from the xml
root = ET.parse(xmlpath).getroot()
image_id_set = set()
for image in root.findall('image'):
    image_id_set.add(int(image.attrib['id']))
mylist = sorted(list(image_id_set))

In [49]:



dfgps

,frame,timestamp,lat,lon
0,0,2020-09-30 23:24:27.693613,13.414324,144.784274
1,1,2020-09-30 23:24:27.760226,13.414324,144.784273
2,2,2020-09-30 23:24:27.826839,13.414324,144.784273
3,3,2020-09-30 23:24:27.893452,13.414324,144.784273
4,4,2020-09-30 23:24:27.960065,13.414324,144.784273
...,...,...,...,...
30310,30310,2020-09-30 23:58:06.733448,13.419328,144.677429
30311,30311,2020-09-30 23:58:06.800061,13.419320,144.677434
30312,30312,2020-09-30 23:58:06.866674,13.419312,144.677438
30313,30313,2020-09-30 23:58:06.933287,13.419303,144.677443


In [50]:
conn = spatialite.connect(db_path)
cur = conn.cursor()

# Get the video_id from the videos field
video = os.path.basename(video_path)
sql = (f'SELECT id FROM videos WHERE name = "{video}"')
cur.execute(sql)
video_id = cur.fetchone()[0]
print(video_id)

# Create a dataframe contining one column, 'frame' (=frame_number)
df = pd.DataFrame(mylist, columns=['frame'])

# Read the gps csv into a second dataframe. Columns are frame, timestamp, lat, and lon. 
dfgps = pd.read_csv(gpspath)

# Merge the two dataframes, joining on the "frame" column.
df = df.merge(dfgps)

1


In [51]:
# Populate the frames table
cur.execute("BEGIN;")
for i, r in df.iterrows():
    s = 'INSERT INTO frames (video_id,frame_number,time,geometry) VALUES ('      
    s += f"{video_id}, {r.frame}, '{r.timestamp}', "
    s += f"TRANSFORM( GeomFromText('POINT({r.lon} {r.lat})', 4326), 3857));\n"
    cur.execute(s)
cur.execute("COMMIT;")

IntegrityError: UNIQUE constraint failed: frames.video_id, frames.frame_number